# Importing Libraries


In [10]:
from torchvision import models
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import gc 
import numpy as np
from scipy.ndimage import rotate
from PIL import Image
import torch
import csv
from sklearn.metrics import classification_report

# Setting up GPU

In [2]:


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
if torch.cuda.device_count() > 1:
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    model = torch.nn.DataParallel(model)
    

Using device: cuda:0


# Augmenting the images

- Undersampled class 4 to 3000 images, and augmented only class 1 (lowest datasize) to 1000 (30%)


In [ ]:
def random_rotation(image):
    angle = np.random.choice([90, 180, 270])
    return rotate(image, angle, axes=(0, 1), reshape=False, mode='reflect')

    
def load_images(folder_path, target_size=None):
    """ Load all images from a folder and resize them if target_size is provided. """
    images = []
    filenames = os.listdir(folder_path)
    for filename in filenames:
        img_path = os.path.join(folder_path, filename)
        with Image.open(img_path) as img:
            if target_size:
                img = img.resize(target_size, Image.Resampling.LANCZOS)
            images.append(np.array(img, dtype=np.float32))
    return images, filenames

def augment_images(data_gen, images, labels, batch_size, save_path, prefix, target_count):
    """ Augment images and save them to a directory, stopping when the target count is reached. """
    generator = data_gen.flow(images, labels, batch_size=batch_size, save_to_dir=save_path, save_prefix=prefix, save_format='jpeg')
    current_count = len(images)  
    for i, (img_batch, label_batch) in enumerate(generator):
        current_count += len(img_batch)  
        if current_count >= target_count:  
            print(f"Target reached with batch {i}. Total count now {current_count}.")
            break
        print(f"Batch {i} saved, {len(img_batch)} images")
        gc.collect()  

def generate_and_save_augmented_images(root_folder, save_folder, target_size=None, target_count=3000):
    data_gen = ImageDataGenerator(
        preprocessing_function=random_rotation,  
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.4,
        shear_range=20.0,
        fill_mode='reflect'
    )
    label_encoder = LabelEncoder()

    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        save_path = os.path.join(save_folder, folder_name)
        if not os.path.exists(save_path):
            os.makedirs(save_path)  # Ensure save directory exists
        
        images, filenames = load_images(folder_path, target_size=target_size)
        labels = [folder_name] * len(images)
        images_array = np.array(images)
        labels_array = label_encoder.fit_transform(labels)
        labels_array = to_categorical(labels_array)

        current_count = len(images)
        needed = target_count - current_count
        print(f"Processing {folder_name}, initial count: {current_count}, target: {target_count}, needed: {needed}")
        
        if needed > 0:
            batch_size = min(100, needed)
            augment_images(data_gen, images_array, labels_array, batch_size, save_path, 'aug', target_count)

root_folder = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev2\\train'
save_folder = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\train'

generate_and_save_augmented_images(root_folder, save_folder, target_size=(224, 224), target_count=1000)


# Labelling with csv

In [4]:


def create_csv(base_path,output_csv_file):
    with open(output_csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Label"])  
    
        for folder in os.listdir(base_path):
            folder_path = os.path.join(base_path, folder)
            
            if os.path.isdir(folder_path):
                for filename in os.listdir(folder_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                        writer.writerow([filename, folder])
                    else:
                        print(filename,folder,"CHECKING!")

train_path = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\train"
train_csv = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\train\\image_labels.csv"
test_path = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\test"
test_csv = "D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\test\\image_labels.csv"

create_csv(train_path,train_csv)
create_csv(test_path,test_csv)


# Setting up model

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

In [7]:
dataset_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\train' 
testing_path = 'D:\\Datasets\\CropDiseaseClassificationOriginal\\split_crop_diseasev3\\test' 
train_dataset = ImageFolder(root=dataset_path, transform=transform)
test_dataset = ImageFolder(root=testing_path, transform=transform)
batch_size = 32
num_workers = 4
pin_memory = True

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)


In [8]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

num_epochs=20

for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')

Epoch 1: Train Loss: 1.1590
Epoch 2: Train Loss: 0.7686
Epoch 3: Train Loss: 0.5290
Epoch 4: Train Loss: 0.3305
Epoch 5: Train Loss: 0.1781
Epoch 6: Train Loss: 0.0858
Epoch 7: Train Loss: 0.0486
Epoch 8: Train Loss: 0.0304
Epoch 9: Train Loss: 0.0179
Epoch 10: Train Loss: 0.0154
Epoch 11: Train Loss: 0.0131
Epoch 12: Train Loss: 0.0171
Epoch 13: Train Loss: 0.0086
Epoch 14: Train Loss: 0.0078
Epoch 15: Train Loss: 0.0075
Epoch 16: Train Loss: 0.0064
Epoch 17: Train Loss: 0.0051
Epoch 18: Train Loss: 0.0046
Epoch 19: Train Loss: 0.0065
Epoch 20: Train Loss: 0.0044


In [11]:
def get_predictions(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['Cassava Bacterial Blight (CBB)', 'Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)','Cassava Mosaic Disease (CMD)','Healthy']))

                                     precision    recall  f1-score   support

     Cassava Bacterial Blight (CBB)       0.58      0.54      0.56       221
Cassava Brown Streak Disease (CBSD)       0.74      0.75      0.74       431
         Cassava Green Mottle (CGM)       0.74      0.72      0.73       493
       Cassava Mosaic Disease (CMD)       0.82      0.84      0.83       528
                            Healthy       0.62      0.64      0.63       466

                           accuracy                           0.72      2139
                          macro avg       0.70      0.70      0.70      2139
                       weighted avg       0.72      0.72      0.72      2139



In [12]:
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model = model.to(device)

num_epochs = 10
for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)  
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')



C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abdul\miniconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1: Train Loss: 1.2645
Epoch 2: Train Loss: 1.0841
Epoch 3: Train Loss: 0.9758
Epoch 4: Train Loss: 0.8946
Epoch 5: Train Loss: 0.8908
Epoch 6: Train Loss: 0.7882
Epoch 7: Train Loss: 0.7336
Epoch 8: Train Loss: 0.6361
Epoch 9: Train Loss: 0.6081
Epoch 10: Train Loss: 0.4891


In [13]:
labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy']))

              precision    recall  f1-score   support

         CBB       0.44      0.24      0.31       221
        CBSD       0.62      0.67      0.64       431
         CGM       0.70      0.61      0.65       493
         CMD       0.63      0.90      0.74       528
     Healthy       0.57      0.45      0.50       466

    accuracy                           0.62      2139
   macro avg       0.59      0.57      0.57      2139
weighted avg       0.61      0.62      0.60      2139



In [14]:
class CustomConvNet(nn.Module):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16 * 54 * 54, 120)  
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, 5)#last layer 5 output classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 16 * 54 * 54)  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [15]:
model = CustomConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs=10

for epoch in range(num_epochs): 
    model.train()  
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  
        optimizer.zero_grad()  
        outputs = model(images)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step()  
        total_loss += loss.item() * images.size(0)
    print(f'Epoch {epoch+1}: Train Loss: {total_loss / len(train_loader.dataset):.4f}')

Epoch 1: Train Loss: 1.5918
Epoch 2: Train Loss: 1.5659
Epoch 3: Train Loss: 1.5442
Epoch 4: Train Loss: 1.4991
Epoch 5: Train Loss: 1.4405
Epoch 6: Train Loss: 1.4008
Epoch 7: Train Loss: 1.3681
Epoch 8: Train Loss: 1.3370
Epoch 9: Train Loss: 1.2830
Epoch 10: Train Loss: 1.2282


In [16]:
labels, preds = get_predictions(model, test_loader)
print(classification_report(labels, preds, target_names=['CBB', 'CBSD', 'CGM', 'CMD', 'Healthy']))

              precision    recall  f1-score   support

         CBB       0.21      0.59      0.31       221
        CBSD       0.43      0.13      0.21       431
         CGM       0.39      0.23      0.29       493
         CMD       0.51      0.34      0.41       528
     Healthy       0.27      0.43      0.33       466

    accuracy                           0.32      2139
   macro avg       0.36      0.35      0.31      2139
weighted avg       0.38      0.32      0.31      2139

